<a href="https://colab.research.google.com/github/auroramugnai/arXiv_classification/blob/main/arXiv_classification/keywords_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone the github repository and move to the inner directory.

In [1]:
!git clone https://github.com/auroramugnai/arXiv_classification.git
%cd arXiv_classification/arXiv_classification

Cloning into 'arXiv_classification'...
remote: Enumerating objects: 652, done.
remote: Counting objects: 100% (350/350), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 652 (delta 209), reused 243 (delta 163), pack-reused 302
Receiving objects: 100% (652/652), 14.92 MiB | 11.68 MiB/s, done.
Resolving deltas: 100% (362/362), done.
/content/arXiv_classification/arXiv_classification


# 1) Build the dataset

In [2]:
import json
import random
import zipfile

import dask.bag as db
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import utils

In [3]:
SEED = 42 # fix random seed for reproducibility

(Or run this to read from .csv 10k articles with already extracted keywords and skip to section 2.)

In [ ]:
# path = f"./kws_cs_10k.csv"
# df = pd.read_csv(path, dtype=str)

## 1.1Download the dataset
The following line of code comes from clicking on "Copy API command" in https://www.kaggle.com/datasets/Cornell-University/arxiv.

In [4]:
!kaggle datasets download -d Cornell-University/arxiv

Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv
License(s): CC0-1.0
100% 1.28G/1.28G [00:57<00:00, 20.7MB/s]
100% 1.28G/1.28G [00:57<00:00, 23.7MB/s]


Unzip the downloaded file.

In [5]:
with zipfile.ZipFile('./arxiv.zip', 'r') as zip_ref:
    zip_ref.extractall()

The unzipping creates an "arxiv-metadata-oai-snapshot.json". We now create a dask bag out of it.

In [6]:
path = "./arxiv-metadata-oai-snapshot.json"
arxiv_data = db.read_text(path).map(json.loads)
arxiv_data.take(1)

({'id': '0704.0001',
  'submitter': 'Pavel Nadolsky',
  'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
  'comments': '37 pages, 15 figures; published version',
  'journal-ref': 'Phys.Rev.D76:013009,2007',
  'doi': '10.1103/PhysRevD.76.013009',
  'report-no': 'ANL-HEP-PR-07-12',
  'categories': 'hep-ph',
  'license': None,
  'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with d

## 1.2 Get rid of some unnecessary information

In [7]:
# Get the latest version of the articles.
get_latest_version = lambda x: x['versions'][-1]['created']

# Only keep articles published after 2022.
is_after_2020 = lambda x: int(get_latest_version(x).split(' ')[3]) > 2020

# Only keep some information.
cut_info = lambda x: {'id': x['id'],
                      'title': x['title'],
                      'category':x['categories'].split(' '),
                      'abstract':x['abstract'],}

# Only keep Computer Science macro-category.
is_only_cs = lambda x: all([s.startswith("cs.") for s in x['categories'].split(' ')])

arxiv_data_filtered = (arxiv_data.filter(is_after_2020).filter(is_only_cs).map(cut_info).compute())


# Create a pandas dataframe and save it to csv.
df = pd.DataFrame(arxiv_data_filtered)
df.to_csv("./cs_arxiv_data_filtered.csv", index=False)
df.head()

,id,title,category,abstract
0,0710.3901,A recursive linear time modular decomposition ...,[cs.DM],A module of a graph G is a set of vertices t...
1,0711.2010,A Polynomial Time Algorithm for Graph Isomorphism,[cs.CC],We claimed that there is a polynomial algori...
2,0802.3414,A Universal In-Place Reconfiguration Algorithm...,"[cs.CG, cs.MA, cs.RO]",In the modular robot reconfiguration problem...
3,0803.3946,On the `Semantics' of Differential Privacy: A ...,"[cs.CR, cs.DB]","Differential privacy is a definition of ""pri..."
4,0805.1877,Perfect tag identification protocol in RFID ne...,[cs.NI],Radio Frequency IDentification (RFID) system...


## 1.3 Get a fixed number of articles
To speed up computation and avoid a session crash.

In [32]:
df = pd.read_csv("cs_arxiv_data_filtered.csv", dtype=str)

In [33]:
num_data = 10000 # number of articles that we want to keep
print(f"The dataset contains {len(df)} articles.")

# Sample the dataset only if its length exceeds num_data.
if(len(df) > num_data):
    n_sample = num_data
    random.seed(SEED)
    df = df.sample(n=n_sample, axis=0)

print(f"The dataset contains {len(df)} articles.")

The dataset contains 199846 articles.
The dataset contains 10000 articles.


# 2) Text-processing

In [12]:
!pip install -U spacy -q
!python -m spacy download en_core_web_md -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [28]:
import en_core_web_md
import spacy
from tqdm import tqdm

Clean out the strings (this step will take a while).

In [34]:
# Remove stop words, punctuation, special characters, numbers.
nlp = spacy.load("en_core_web_md")
tqdm.pandas() # to display progress bar

# First on abstracts.
abs_cleaner = lambda x: utils.text_cleaner(text=x["abstract"], nlp=nlp)
df["clean_abstract"] = df.progress_apply(abs_cleaner, axis=1)

# Then on titles.
tit_cleaner = lambda x: utils.text_cleaner(text=x["title"], nlp=nlp)
df["clean_title"] = df.progress_apply(tit_cleaner, axis=1)

df.tail()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
100%|██████████| 10000/10000 [01:28<00:00, 112.37it/s]


,id,title,category,abstract,clean_abstract,clean_title
56022,2112.11255,Mind the Gap! A Study on the Transferability o...,"['cs.SE', 'cs.AI', 'cs.RO']",Safe deployment of self-driving cars (SDC) n...,safe deployment self drive car sdc necessitate...,mind gap study transferability virtual vs phys...
171960,2312.11932,Unravelling Expressive Delegations: Complexity...,"['cs.GT', 'cs.CC', 'cs.DS']",We consider binary group decision-making und...,consider binary group decision making rich mod...,unravel expressive delegations complexity norm...
64042,2203.00860,D^2ETR: Decoder-Only DETR with Computationally...,['cs.CV'],DETR is the first fully end-to-end detector ...,detr fully end end detector predict final set ...,decoder detr computationally efficient cross s...
117270,2303.05275,Detecting Images Generated by Diffusers,['cs.CV'],This paper explores the task of detecting im...,paper explore task detect image generate text ...,detect image generate diffuser
10908,2101.07223,Leveraging AI to optimize website structure di...,"['cs.CR', 'cs.AI', 'cs.LG']",Dirbusting is a technique used to brute forc...,dirbusting technique use brute force directory...,leverage ai optimize website structure discove...


In [35]:
# Add a space to separate title and abstract.
df["clean_text"] = df["clean_title"] + " " + df["clean_abstract"]
df.head()

,id,title,category,abstract,clean_abstract,clean_title,clean_text
132689,2305.18888,Contrastive Shapelet Learning for Unsupervised...,['cs.LG'],Recent studies have shown great promise in u...,recent study great promise unsupervised repres...,contrastive shapelet learning unsupervised mul...,contrastive shapelet learning unsupervised mul...
24466,2104.14854,NTIRE 2021 Challenge on Image Deblurring,['cs.CV'],Motion blur is a common photography artifact...,motion blur common photography artifact dynami...,ntire challenge image deblurring,ntire challenge image deblurring motion blur c...
105073,2212.03955,Experiences from the MediaEval Predicting Medi...,"['cs.CV', 'cs.AI']",The Predicting Media Memorability task in th...,predict media memorability task mediaeval eval...,experience mediaeval predict media memorabilit...,experience mediaeval predict media memorabilit...
170448,2312.07537,FreeInit: Bridging Initialization Gap in Video...,['cs.CV'],Though diffusion-based video generation has ...,diffusion base video generation witness rapid ...,freeinit bridging initialization gap video dif...,freeinit bridging initialization gap video dif...
39400,2108.09537,Using growth transform dynamical systems for s...,"['cs.SD', 'cs.HC', 'cs.LG']","Sonification, or encoding information in mea...",sonification encode information meaningful aud...,use growth transform dynamical system spatio t...,use growth transform dynamical system spatio t...


# 4) Keywords extraction

In [37]:
!pip install KeyBERT -q
!pip install keyphrase-vectorizers -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.5/363.5 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.8/772.8 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 88.9 MB/s eta 0:00:00


In [38]:
from keybert import KeyBERT
from sklearn.feature_extraction.text import TfidfVectorizer

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [39]:
kw_model = KeyBERT('all-mpnet-base-v2')

extraction = lambda x: utils.extract_kws(text=x["clean_text"],
                                         kw_model=kw_model,
                                         seed=x["clean_title"].split(" "))

df["keywords"] = df.progress_apply(extraction, axis=1)
df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 10000/10000 [18:26<00:00,  9.04it/s]


,id,title,category,abstract,clean_abstract,clean_title,clean_text,keywords
132689,2305.18888,Contrastive Shapelet Learning for Unsupervised...,['cs.LG'],Recent studies have shown great promise in u...,recent study great promise unsupervised repres...,contrastive shapelet learning unsupervised mul...,contrastive shapelet learning unsupervised mul...,"[shapelet, multivariate, series, contrasting]"
24466,2104.14854,NTIRE 2021 Challenge on Image Deblurring,['cs.CV'],Motion blur is a common photography artifact...,motion blur common photography artifact dynami...,ntire challenge image deblurring,ntire challenge image deblurring motion blur c...,"[deblurring, artifact, jpeg, ntire]"
105073,2212.03955,Experiences from the MediaEval Predicting Medi...,"['cs.CV', 'cs.AI']",The Predicting Media Memorability task in th...,predict media memorability task mediaeval eval...,experience mediaeval predict media memorabilit...,experience mediaeval predict media memorabilit...,"[memorability, mediaeval, resource, predict]"
170448,2312.07537,FreeInit: Bridging Initialization Gap in Video...,['cs.CV'],Though diffusion-based video generation has ...,diffusion base video generation witness rapid ...,freeinit bridging initialization gap video dif...,freeinit bridging initialization gap video dif...,"[denoise, generate, video, freeinit]"
39400,2108.09537,Using growth transform dynamical systems for s...,"['cs.SD', 'cs.HC', 'cs.LG']","Sonification, or encoding information in mea...",sonification encode information meaningful aud...,use growth transform dynamical system spatio t...,use growth transform dynamical system spatio t...,"[sonification, eeg, data, transform]"


# 4) Classification
Given an article:

- its feature X will be the cleaned text
- its label y will be its keyword

In [ ]:
!pip install scikit-multilearn -q

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [ ]:
# Preparing X (features)
X = df["text"]

# Preparing y (labels)
y = df['keywords']

#Split data into train/test.
X_train, X_test, y_train_tot, y_test_tot = train_test_split(X, y,
                                                            test_size=0.5,
                                                            random_state=SEED)

In [ ]:
# Select only the first keyword for every article.
y_train = [eval(x)[0] for x in y_train_tot]
y_test = [eval(x)[0] for x in y_test_tot]

Do the classification.

In [ ]:
model = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                  ('svm_model', LinearSVC(verbose=1))])

y_pred = utils.run_model(model, X_train, X_test, y_train, y_test,
                         multilabel=False)

print('accuracy: ', accuracy_score(y_test, y_pred))

df_pred = pd.DataFrame({'clean_text': X_test,
                        'true_kws': y_test_tot,
                        'first_true_kw': y_test,
                        'predicted_kw': y_pred})

[LibLinear]accuracy:  0.371


In [ ]:
df_pred = df_pred.reset_index(drop=True)
df_pred.head()

,clean_text,true_kws,first_true_kw,predicted_kw
0,building defect prediction models by online le...,"['learning', 'predict', 'defect', 'auc']",learning,prediction
1,adaptive discretization use voronoi trees for ...,"['pomdp', 'discretization', 'voronoi', 'tree']",pomdp,action
2,study the explanation for the automate predict...,"['classification', 'bug', 'shap', 'understand']",classification,explainability
3,airtrack onboard deep learning framework for l...,"['aircraft', 'tracking', 'dataset', 'daa']",aircraft,tracking
4,query complexity based optimal processing of r...,"['workload', 'query', 'partition', 'dataset']",workload,rdf


In [ ]:
# Get the number of predicted kws that are contained in the list of true kws.
is_in_true_kws = lambda x: x.predicted_kw in x.true_kws
num_true = df_pred.apply(is_in_true_kws, axis=1).value_counts().loc[True]

# Turn it to percentage.
print(f"{round((num_true/len(df_pred))*100, 2)}% of predicted kws are true kws")

57.14% of predicted kws are true kws


# 3) Compute the distance between the true and the predicted keywords

In [ ]:
import nltk
import spacy
from gensim.models import Word2Vec

In [ ]:
# Lists of the keywords on which we want to compute the similarity.
kws_pred = df_pred['predicted_kw'].values
kws_true = df_pred['first_true_kw'].values

In [ ]:
# Create the corpus using our processed texts.
corpus = list(df['text'].values)

# Tokenize the corpus.
nltk.download('punkt')
tokenized_corpus = [nltk.word_tokenize(text.lower()) for text in corpus]

# Train the Word2Vec model on the created corpus.
model = Word2Vec(tokenized_corpus, min_count=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Compute the meaning similarity.

In [ ]:
simil_meaning_list = [] # meaninig similarity

for i, (kp, kb) in enumerate(zip(kws_pred, kws_true)):
    sim = model.wv.similarity(kp, kb)
    sim = float("{0:.2f}".format(sim))
    if (i<5): print(f"The similarity between '{kp}' and '{kb}' is: {sim}")
    simil_meaning_list.append(sim)

print(f"\nMEAN OF SIMILARITIES: {np.mean(simil_meaning_list)}")

The similarity between 'prediction' and 'learning' is: 0.22
The similarity between 'action' and 'pomdp' is: 0.16
The similarity between 'explainability' and 'classification' is: 0.26
The similarity between 'tracking' and 'aircraft' is: 0.41
The similarity between 'rdf' and 'workload' is: 0.4

MEAN OF SIMILARITIES: 0.595884
